In [1]:
import os, warnings
warnings.simplefilter('ignore')
import ipyleaflet
import time


import ee
ee.Initialize()
import geemap


import requests
import boto3
from google.cloud import storage

import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
import glob

In [2]:
def extract_gee_data(product_path, band_name, band_scale, band_offset, boundary_path, start_date, end_date, reducer = 'mean'):
    
    '''
    product_path: Link to GEE product
    band_name: Name of required band as mentioned in GEE product
    band_scale: Scale factor for the selected band as mentioned in GEE or as per requirement
    band_offset: Offset factor for the selected band as mentioned in GEE or as per requirement
    boundary_path: Path to geojson of area of interest
    start_date: Starting date of period for which data is required
    end_date: Ending date of period for which data is required
    reducer: The type reducer for time series image collection. Can choose between mean, max and min. Default is mean.
    '''
    
    gdf = gpd.read_file(boundary_path)
    fc = ee.FeatureCollection(geemap.geopandas_to_ee(gdf))
    
    if reducer == 'mean':
        ic = ee.ImageCollection(product_path).filterBounds(fc).filterDate(start_date, end_date).select(band_name).reduce(ee.Reducer.mean())
        
    elif reducer == 'max':
        ic = ee.ImageCollection(product_path).filterBounds(fc).filterDate(start_date, end_date).select(band_name).reduce(ee.Reducer.max())
        
    elif reducer == 'min':
        ic = ee.ImageCollection(product_path).filterBounds(fc).filterDate(start_date, end_date).select(band_name).reduce(ee.Reducer.min())
    
    def convert_ic(raw):
        converted = raw.multiply(band_scale).add(band_offset)
        return converted
    
    img = ee.ImageCollection(ic).map(convert_ic)
    img = img.toBands()
    
    return img

In [3]:
def extract_modisterra_lst(boundary_path, start_date, end_date):
    
    '''
    boundary_path: Path to geojson of area of interest
    start_date: Starting date of period for which data is required
    end_date: Ending date of period for which data is required
    '''
    
    product_path = 'MODIS/006/MYD11A1'
    day_band_name = 'LST_Day_1km'
    night_band_name = 'LST_Night_1km'
    band_scale = 0.02
    KtoC = -273.15
    
    lst_day_ic = extract_gee_data(product_path, day_band_name, band_scale, KtoC, boundary_path, start_date, end_date)
    lst_night_ic = extract_gee_data(product_path, night_band_name, band_scale, KtoC, boundary_path, start_date, end_date)
    
    return (lst_day_ic, lst_night_ic)

In [4]:
def export_img_to_drive(img, boundary_path, name, folder, scale):
    
    '''
    img: Image to be exported
    boundary_path: Path to geojson of area of interest
    name: Save name to be given to the file
    folder: folder in google drive where file is to be saved
    scale: Scale/spatial resolution in meters at which file is to be saved. MODIS LST 1000m. Landsat 8 30m.
    '''

    gdf = gpd.read_file(boundary_path)
    fc = ee.FeatureCollection(geemap.geopandas_to_ee(gdf))

    geemap.ee_export_image_to_drive(
            img, 
            description = name,
            folder = folder, 
            scale = scale, 
            region = fc.geometry(),
            maxPixels = 5000000000
        )

In [5]:
#boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/heat/coolmap/data/boundary/boundary-IND-Ahmedabad-wards-Census2011.geojson'
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/heat/coolmap/data/boundary/boundary-ARG-Buenos_Aires-city-ADM2.geojson'

In [6]:
start_date = '2000-02-24'
end_date = '2023-11-30'

In [7]:
lst_day, lst_night = extract_modisterra_lst(boundary_path, start_date, end_date)

In [8]:
export_img_to_drive(lst_day, boundary_path, 'Buenos_Aires_LST_Day', 'gee_exports', scale = 1000)
export_img_to_drive(lst_night, boundary_path, 'Buenos_Aires_LST_Night', 'gee_exports', scale = 1000)